In [61]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from tensorflow import keras

from sklearn.model_selection import train_test_split

In [62]:
data = pd.read_csv('/content/drive/MyDrive/Assignments/NLP - Social Media/Labelled_text.csv')

In [63]:
data.head(10)

,text,label
0,Manchester MMA fighter Brendan Loughnane 21-3-...,mma
1,Former PFL women s lightweight champion Kayla ...,mma
2,Kayla Harrison is a perfectionist inside the c...,mma
3,Ciryl Gane stopped Derrick Lewis with a flurry...,mma
4,As Manchester's featherweight contender Brenda...,mma
5,The culmination of the nearly two-year Bellato...,mma
6,Bellator will be returning to London on Octobe...,mma
7,"Conor McGregor says he had ""multiple stress fr...",mma
8,As Conor McGregor recovers from a horrific and...,mma
9,"Conor McGregor said he was feeling ""tremendous...",mma


In [64]:
data.shape

(21594, 2)

In [65]:
data = data[['text', 'label']]

In [66]:
data.head(10)

,text,label
0,Manchester MMA fighter Brendan Loughnane 21-3-...,mma
1,Former PFL women s lightweight champion Kayla ...,mma
2,Kayla Harrison is a perfectionist inside the c...,mma
3,Ciryl Gane stopped Derrick Lewis with a flurry...,mma
4,As Manchester's featherweight contender Brenda...,mma
5,The culmination of the nearly two-year Bellato...,mma
6,Bellator will be returning to London on Octobe...,mma
7,"Conor McGregor says he had ""multiple stress fr...",mma
8,As Conor McGregor recovers from a horrific and...,mma
9,"Conor McGregor said he was feeling ""tremendous...",mma


In [67]:
data.dropna(inplace=True)

In [68]:
data['label'].value_counts()

other_sport    7686
nonsport       2336
football       2225
soccer         2075
baseball       1965
cricket        1044
basketball      928
rugby           866
boxing          563
tennis          491
f1              458
netball         319
golf            276
hockey          229
mma             116
Name: label, dtype: int64

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data.text, data.label, test_size=0.2, random_state=42, stratify=data.label)

In [70]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42, stratify=y_train)

In [71]:
print(X_train.shape, X_val.shape, X_test.shape)

(12945,) (4316,) (4316,)


In [72]:
y_train_ohe = pd.get_dummies(y_train)
y_test_ohe = pd.get_dummies(y_test)
y_val_ohe = pd.get_dummies(y_val)

In [73]:
embedding = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

In [74]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(y_train_ohe.shape[1], activation='softmax'))

In [75]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_2 (KerasLayer)   (None, 128)               124642688 
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_5 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 15)                975       
Total params: 124,651,919
Trainable params: 124,651,919
Non-trainable params: 0
_________________________________________________________________


In [76]:
model.compile(optimizer='Nadam', loss='categorical_crossentropy', metrics=['accuracy'])

In [77]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [78]:
class_weight = {}
for i in range(y_train_ohe.shape[1]):
  class_weight[i] = y_train_ohe.shape[0]/np.sum(y_train_ohe.iloc[:,i])/y_train_ohe.shape[1]

In [79]:
class_weight

{0: 0.7319762510602206,
 1: 1.5521582733812949,
 2: 2.5608308605341246,
 3: 1.378594249201278,
 4: 3.138181818181818,
 5: 0.6464419475655431,
 6: 5.198795180722892,
 7: 6.299270072992701,
 8: 12.328571428571427,
 9: 4.518324607329843,
 10: 0.615549215406562,
 11: 0.18716113641292562,
 12: 1.6596153846153847,
 13: 0.6931726907630522,
 14: 2.9254237288135596}

In [80]:
history=model.fit(X_train, y_train_ohe, batch_size=32, validation_data=(X_val, y_val_ohe), epochs=20, verbose=1,
                  callbacks=[callback],class_weight=class_weight)

Epoch 1/20
405/405 [==============================] - 30s 71ms/step - loss: 2.3239 - accuracy: 0.2743 - val_loss: 2.0080 - val_accuracy: 0.4780
Epoch 2/20
405/405 [==============================] - 29s 71ms/step - loss: 1.4519 - accuracy: 0.5475 - val_loss: 1.6366 - val_accuracy: 0.5111
Epoch 3/20
405/405 [==============================] - 29s 71ms/step - loss: 0.7810 - accuracy: 0.7122 - val_loss: 1.3647 - val_accuracy: 0.5755
Epoch 4/20
405/405 [==============================] - 29s 71ms/step - loss: 0.4209 - accuracy: 0.8228 - val_loss: 1.3213 - val_accuracy: 0.5927
Epoch 5/20
405/405 [==============================] - 29s 71ms/step - loss: 0.2437 - accuracy: 0.8853 - val_loss: 1.3491 - val_accuracy: 0.5994
Epoch 6/20
405/405 [==============================] - 29s 70ms/step - loss: 0.1519 - accuracy: 0.9229 - val_loss: 1.3498 - val_accuracy: 0.6175
Epoch 7/20
405/405 [==============================] - 29s 71ms/step - loss: 0.0961 - accuracy: 0.9499 - val_loss: 1.4269 - val_accuracy:

In [81]:
results = model.evaluate(X_test, y_test_ohe, verbose=2)

135/135 - 1s - loss: 1.6299 - accuracy: 0.6221


In [82]:
model.evaluate(X_val, y_val_ohe, verbose=2)

135/135 - 1s - loss: 1.5898 - accuracy: 0.6226


[1.5898078680038452, 0.6225671768188477]

In [83]:
model.evaluate(X_train, y_train_ohe, verbose=2)

405/405 - 4s - loss: 0.0459 - accuracy: 0.9910


[0.045941758900880814, 0.9910390377044678]